# Load the Data

In [1]:
import pandas as pd
import numpy as np
import chess

# Load white and black datasets
white_df = pd.read_csv("white_df_100.csv")
black_df = pd.read_csv("black_df_100.csv")


# Pre-process Data

In [2]:
# Concatenate the data
all_df = pd.concat([black_df, white_df])

# Pre-process data
all_df['Is_Check'] = all_df['Is_Check'].astype(int)
all_df['Is_Capture'] = all_df['Piece_Captured'].apply(lambda x: 1 if x != None else 0)
all_df = all_df.drop(columns=['FEN'])

# Save the combined dataset to a file
all_df.to_csv('all_df_100.csv', index=False)


# Define Helper Functions

In [3]:
import ast

# Define helper functions for converting chess moves to tuples and vice versa
def chess_square_to_number(square: str) -> int:
    file = square[0]
    rank = int(square[1])
    file_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8}
    return (file_dict[file] - 1) * 8 + rank

def chess_move_to_tuple(move):
    start_square = chess_square_to_number(move[:2])
    end_square = chess_square_to_number(move[2:])
    return (start_square, end_square)

def chess_moves_to_tuples(moves):
    result = []
    for move in moves:
        result.append(chess_move_to_tuple(move))
    return result


# Convert Moves to Tuples

In [4]:
# Convert moves to tuples
all_df["Move"] = all_df["Move"].apply(lambda x: chess_move_to_tuple(x))
all_df["Top_10_Moves"] = all_df["Top_10_Moves"].apply(lambda x: ast.literal_eval(x))
all_df["Top_10_Moves"] = all_df["Top_10_Moves"].apply(lambda x: chess_moves_to_tuples(x))


# Prepare Data for Model Training

In [5]:
# Prepare data for model training
candidate_data = all_df["Top_10_Moves"].tolist()
target_data = all_df["Move"].tolist()


# Calculate Euclidean Distances

In [6]:
# Calculate the Euclidean distance between each candidate move and the target move
X, y = [], []
for i, candidates in enumerate(candidate_data):
    target = target_data[i]
    for candidate in candidates:
        X.append(candidate)
        y.append(np.linalg.norm(np.array(candidate) - np.array(target)))

X = np.array(X)
y = np.array(y)


# Train-Test Split

In [7]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train the Model

In [8]:
from sklearn.ensemble import RandomForestRegressor

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

# Evaluate the Model

In [9]:
# Evaluate the model
y_pred = model.predict(X_test)


# Define Helper Functions for Sorting Candidates and Calculating Accuracy

In [10]:
import random

def sort_candidates(candidates):
    likelihoods = model.predict(candidates)
    return sorted(candidates, key=lambda x: likelihoods[candidates.index(x)], reverse=True)

def get_top_candidate(candidates):
    likelihoods = model.predict(candidates)
    sorted_candidates = sorted(candidates, key=lambda x: likelihoods[candidates.index(x)], reverse=False)
    return sorted_candidates[0]

def get_random_candidate(candidates):
    return random.choice(candidates)


# Calculate Model Accuracy

In [11]:
# Calculate the accuracy of the model by comparing the top predicted move to the target move
correct = sum(get_top_candidate(candidates) == target for candidates, target in zip(candidate_data, target_data))
total = len(candidate_data)
accuracy = correct / total
print("Pure Accuracy:", accuracy)


Pure Accuracy: 0.237831701433234


# Calculate Random Selection Accuracy

In [12]:
# Calculate the accuracy of randomly selecting a move from the candidates
random_correct = sum(get_random_candidate(candidates) == target for candidates, target in zip(candidate_data, target_data))
random_accuracy = random_correct / total
print("Random Selection Accuracy:", random_accuracy)


Random Selection Accuracy: 0.11338158081453101


# Compare Accuracies

In [13]:
print("Trained Model Pure Accuracy:", accuracy)
print("Random Selection Accuracy:", random_accuracy)


Trained Model Pure Accuracy: 0.237831701433234
Random Selection Accuracy: 0.11338158081453101


# Save the Model

In [14]:
import joblib

# Save the model to a file
joblib.dump(model, 'final_model_chess.joblib')


['final_model_chess.joblib']